# 编程和测试要求
## 3. 拓展功能

> 考虑到向实用性扩展，加密算法的数据输入可以是ASII编码字符串(分组为2 Byte)，对应地输出也可以是ACII字符串(很可能是乱码)。

我们这里选择对加密结果按照16进制的格式输出，避免输出乱码或者无法显示的系统控制符：

In [14]:
# 加载实现的加密算法类以及测试用类
from SAES.SAES import SAES
from unittest.mock import Mock

### 3.1 ASCII码的处理

在后端的视图函数中根据用户的选择处理不同格式的输入：

In [15]:
# 用户明文处理函数
def plaintext_sent(request):
    if request.method == 'POST':
        key_input = request.POST['key']
        plaintext_input = request.POST['plaintext']
        type = request.POST['type']
        if type == "ASCII":
            saes = SAES(key=key_input)
            out = ""

            # 转换输入为二进制
            binary_input = ''.join([bin(ord(char)).replace("0b", "").zfill(8) for char in plaintext_input])

            # 检查字符数并进行填充
            if len(plaintext_input) % 2 == 1:  # 奇数字符，添加00000001
                binary_input += '00000001'
            else:  # 偶数字符，添加00000010 00000010
                binary_input += '00000010' + '00000010'

            for i in range(0, len(binary_input), 16):
                bin_block = binary_input[i:i + 16]
                out_hex = hex(int(saes.encrypt(bin_block), 2))[2:].zfill(4)  # 加密并转换为16进制
                out += out_hex

            data = {
                "ciphertext": out
            }
            return data

        elif type == "Bit":
            saes = SAES(key=key_input)
            out_bin = saes.encrypt(plaintext_input)
            out_hex = hex(int(out_bin, 2))[2:].zfill(4)
            data = {
                "ciphertext": out_bin
            }
            return data


考虑到用户的输入分组并非是2bytes,处理并填充ASCII字符是至关重要的。这里我们采用PKCS#7标准来进行填充：

为奇数长度的输入加上00000001，为偶数长度的输入加上00000010 00000010。
这意味着解密结果都将是偶数位，并且后8bit只有两种可能。再分别进行删除，这就避免了二义性，不会有两个不同的输入在加密后产生相同输出。

我们将模拟一个用户在前端生成的HttpRequest请求来测试:

In [16]:
# 创建一个模拟的 HttpRequest 对象
request_plain = Mock()

# 设置模拟对象的属性
request_plain.method = 'POST'
request_plain.POST = {
    'key': '1100010111011101',
    'plaintext': 'code',
    'type': 'ASCII',
}
# 测试
response = plaintext_sent(request_plain)
print("'code'加密后的密文是:"+response['ciphertext'])

'code'加密后的密文是:c4cc5136319a


### 3.2 对密文的处理

在后端的视图函数中根据用户的选择处理不同格式的输入：

In [17]:
# 用户输入待转换的密文
def ciphertext_sent(request):
    #
    # 用户输入的文本就记作：plaintext
    if request.method == 'POST':
        key_input = request.POST['key']
        ciphertext_input = request.POST['plaintext']
        type = request.POST['type']

        # 实例化SAES对象
        saes = SAES(key=key_input)

        if type == "ASCII":
            out = ""

            # 每4个字符（16位）解密一次
            for i in range(0, len(ciphertext_input), 4):
                hex_block = ciphertext_input[i:i + 4]
                bin_block = bin(int(hex_block, 16)).replace("0b", "").zfill(16)
                out_bin = saes.decrypt(bin_block)

                # 从二进制块中检索原始字符
                out_chr1 = chr(int(out_bin[:8], 2))
                out_chr2 = chr(int(out_bin[8:], 2))

                # 检查填充字符并按需要删除它们
                if out_bin[8:] == '00000001':
                    out += out_chr1
                elif out_bin[:8] == '00000010' and out_bin[8:] == '00000010':
                    continue
                else:
                    out += out_chr1 + out_chr2

            data = {
                "ciphertext": out
            }
            return data

        elif type == "Bit":
            bin_input = bin(int(ciphertext_input,2)).replace("0b", "").zfill(16)
            out_bin = saes.decrypt(bin_input)
            data = {
                "ciphertext": out_bin
            }
            return data

In [18]:
# 创建一个模拟的 HttpRequest 对象并输入
request_cipher = Mock()

# 设置模拟对象的属性
request_cipher.method = 'POST'
request_cipher.POST = {
    'key': '1100010111011101',
    'plaintext': 'c4cc5136319a',
    'type': 'ASCII',
}

#函数测试
response = ciphertext_sent(request_cipher)
print("解密后的明文为:"+response["ciphertext"])

解密后的明文为:code


经过一次模拟的用户请求测试，可以看出我们对ASCII码的处理无误。